# Predict Future Sales (Kaggle playground)

See [Predict Future Sales on Kaggle](https://www.kaggle.com/c/competitive-data-science-predict-future-sales).

I accepted the terms & conditions then downloaded the data.
`kaggle competitions download -c competitive-data-science-predict-future-sales`

'Prelude' copied from lesson3-rossmann:

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import functools
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
from IPython.display import HTML

PATH='data/predict-future-sales/'

In [2]:
!ls {PATH}

item_categories.csv  sales_train.csv	       shops.csv
items.csv	     sample_submission.csv.gz  test.csv.gz


## Import data

In [3]:
table_names = ['item_categories', 'items', 'shops', 'sales_train', 'test']

def load_table(table, root_path):
    fname = os.path.join(root_path, f"{table}.csv")
    if os.path.exists(fname):
        compression = None
    else:
        fname = f"{fname}.gz"
        compression = 'gzip'
    
    return pd.read_csv(fname, compression=compression)

tables = [load_table(t, root_path=PATH) for t in table_names]

In [4]:
for t in tables: display(t.head())

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
for t in tables:
    display(DataFrameSummary(t).summary())

,item_category_name,item_category_id
count,NaN,84
mean,NaN,41.5
std,NaN,24.3926
min,NaN,0
25%,NaN,20.75
50%,NaN,41.5
75%,NaN,62.25
max,NaN,83
counts,84,84
uniques,84,84


,item_name,item_id,item_category_id
count,NaN,22170,22170
mean,NaN,11084.5,46.2908
std,NaN,6400.07,15.9415
min,NaN,0,0
25%,NaN,5542.25,37
50%,NaN,11084.5,40
75%,NaN,16626.8,58
max,NaN,22169,83
counts,22170,22170,22170
uniques,22170,22170,84


,shop_name,shop_id
count,NaN,60
mean,NaN,29.5
std,NaN,17.4642
min,NaN,0
25%,NaN,14.75
50%,NaN,29.5
75%,NaN,44.25
max,NaN,59
counts,60,60
uniques,60,60


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,NaN,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06
mean,NaN,14.5699,33.0017,10197.2,890.853,1.24264
std,NaN,9.42299,16.227,6324.3,1729.8,2.61883
min,NaN,0,0,0,-1,-22
25%,NaN,7,22,4476,249,1
50%,NaN,14,31,9343,399,1
75%,NaN,23,47,15684,999,1
max,NaN,33,59,22169,307980,2169
counts,2935849,2935849,2935849,2935849,2935849,2935849
uniques,1034,34,60,21807,19993,198


,ID,shop_id,item_id
count,214200,214200,214200
mean,107100,31.6429,11019.4
std,61834.4,17.5619,6252.64
min,0,2,30
25%,53549.8,16,5381.5
50%,107100,34.5,11203
75%,160649,47,16071.5
max,214199,59,22167
counts,214200,214200,214200
uniques,214200,42,5100


In [6]:
item_categories, items, shops, sales_train, test = tables
sales_train.memory_usage(), sales_train.shape, len(pd.unique(sales_train['item_id'])) * len(pd.unique(sales_train['shop_id'])) * sales_train['date_block_num'].max()

(Index                   80
 date              23486792
 date_block_num    23486792
 shop_id           23486792
 item_id           23486792
 item_price        23486792
 item_cnt_day      23486792
 dtype: int64, (2935849, 6), 43177860)

 The mission is to predict next month's sales data, hence add a 'date_block_num' column to the test data representing the month following the last month for which data is available and assign a dummy date to be able to use `add_datepart` (ideally the formula for `test['date']` would parse from `sales_train` what `date_block_num==0` means, but I kept it simple):

In [7]:
next_month = 1 + sales_train['date_block_num'].max()
test['date_block_num'] = next_month
test['date'] = "01.{m}.{y}".format(m=next_month % 12 + 1, y=2013 + next_month // 12)

Since we're interested in the prediction of a month's sales, remove all time information that is finer grained than a month (see the `drop` in `do_date_transfo`).

In [8]:
def reformat_ddmmyyyy_to_yyyymmdd(x):
    return "{2}-{1}-{0}".format(*(x.split('.')))

def do_date_transfo(x):
    x['date'] = x['date'].apply(reformat_ddmmyyyy_to_yyyymmdd)
    add_datepart(x, 'date', drop=True)
    x.drop(columns=['Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 'Elapsed', 
                    # These look like month-granular, but e.g. 2-Jan is not seen as year_start, so drop these as well
                    'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'], inplace=True)
    return x

sales_train = do_date_transfo(sales_train)
test = do_date_transfo(test)
test.head(2)

,ID,shop_id,item_id,date_block_num,Year,Month
0,0,5,5037,34,2015,11
1,1,5,5320,34,2015,11


Copied from lesson3-rossmann:

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join (i.e. inner join + keep rows of left table that don't match anything in the right table) of right on the left argument using the given fields for each table.

Pandas does joins using the merge method. The suffixes argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a `"_y"` to those on the right.


In [9]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    """@param left: Dataframe
    @param right: Dataframe
    @param left_on: column name in left table
    @param right_on: (default: left_on) column name in right table
    @param suffix: (default: "_y") appended to duplicate column names from the right table"""
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

def denormalize(t):
    """Denormalize table by adding shop names, item names & item categories
    
    This function is specific to the data model of this Kaggle competition."""
    t = join_df(t, shops, 'shop_id')
    t = join_df(t, items, 'item_id')
    t = join_df(t, item_categories, 'item_category_id')
    return t

### Aggregate training data per month.

First, enrich the sales data with all other tables we have, so that we can e.g. group by categories instead of items later.

In [10]:
sales_train = denormalize(sales_train)
test = denormalize(test)
sales_train.head(3)

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month,shop_name,item_name,item_category_id,item_category_name
0,0,59,22154,999.0,1.0,2013,1,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,25,2552,899.0,1.0,2013,1,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,25,2552,899.0,-1.0,2013,1,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил


Introduce a new column representing the revenue brought by a product: `gross = item_cnt_day * item_price`:

In [11]:
sales_train['gross'] = sales_train['item_price'] * sales_train['item_cnt_day']

Looking at the data, it seemed like at least one shop was open every day, but there are shops that were not open every day.  Record per shop how many items (all lumped together) they sold and what revenue they generated.

In [12]:
sales_and_monthly_revenue = sales_train.groupby(['date_block_num', 'shop_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
sales_and_monthly_revenue.rename(inplace=True, columns={
    'item_cnt_day': 'all_shop_items_sold',
    'gross': 'all_shop_gross'
});
sales_and_monthly_revenue.head(5)

,date_block_num,shop_id,all_shop_items_sold,all_shop_gross
0,0,0,5578.0,2966412.00
1,0,1,2947.0,1527320.00
2,0,2,1146.0,1085064.80
3,0,3,767.0,560585.01
4,0,4,2114.0,1433666.00


How many items of each sort were sold in all stores for each time period and what revenue did each item generate?

In [13]:
items_globally = sales_train.groupby(['date_block_num', 'item_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
items_globally.rename(inplace=True, columns={
    'item_cnt_day': 'global_sold',
    'gross': 'global_gross'
});
items_globally.head(5)

,date_block_num,item_id,global_sold,global_gross
0,0,19,1.0,28.00
1,0,27,7.0,16275.00
2,0,28,8.0,4392.00
3,0,29,4.0,9387.00
4,0,32,299.0,101515.86


How many items of each sort were sold per store for each time period and what revenue did they generate?

In [14]:
items_per_shop = sales_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'Month': 'min', # should be the same for all anyway and we're going to need it because I assume there's a yearly component
    'item_price': 'mean', # actually I hope it's constant inside a month...
    'gross': 'sum'})
items_per_shop.head(5)

,date_block_num,shop_id,item_id,item_cnt_day,Month,item_price,gross
0,0,0,32,6.0,1,221.0,1326.0
1,0,0,33,3.0,1,347.0,1041.0
2,0,0,35,1.0,1,247.0,247.0
3,0,0,43,1.0,1,221.0,221.0
4,0,0,51,2.0,1,128.5,257.0


How much (count and revenue) was sold per category for each time period (globally & per store)?

In [15]:
cats_globally = sales_train.groupby(['date_block_num', 'item_category_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
cats_globally.rename(inplace=True, columns={
    'item_cnt_day': 'global_cat_sold',
    'gross': 'global_cat_gross'
});
cats_per_shop = sales_train.groupby(['date_block_num', 'shop_id', 'item_category_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
cats_per_shop.rename(inplace=True, columns={
    'item_cnt_day': 'cat_shop_sold',
    'gross': 'cat_shop_gross'
});
cats_globally.head(5)

,date_block_num,item_category_id,global_cat_sold,global_cat_gross
0,0,0,1.0,148.00
1,0,1,1.0,148.00
2,0,2,1390.0,2848554.48
3,0,3,440.0,209160.13
4,0,4,251.0,232099.00


Paste together all the cumulative data, the goal is a table with unique (date_block_num, shop_id, item_id) rows.  Start from `items_per_shop` then join other cumulative data onto it.

In [16]:
 data = functools.reduce(
    lambda src, extra: join_df(src, *extra),
    [[items, 'item_id'],
     [item_categories, 'item_category_id'],
     [shops, 'shop_id'],
     [sales_and_monthly_revenue, ('date_block_num', 'shop_id')],
     [items_globally, ('date_block_num', 'item_id')],
     [cats_globally, ('date_block_num', 'item_category_id')],
     [cats_per_shop, ('date_block_num', 'shop_id', 'item_category_id')]],
    items_per_shop)

In [17]:
data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_day', 'Month',
       'item_price', 'gross', 'item_name', 'item_category_id',
       'item_category_name', 'shop_name', 'all_shop_items_sold',
       'all_shop_gross', 'global_sold', 'global_gross', 'global_cat_sold',
       'global_cat_gross', 'cat_shop_sold', 'cat_shop_gross'],
      dtype='object')

In [18]:
data.head(10)

,date_block_num,shop_id,item_id,item_cnt_day,Month,item_price,gross,item_name,item_category_id,item_category_name,shop_name,all_shop_items_sold,all_shop_gross,global_sold,global_gross,global_cat_sold,global_cat_gross,cat_shop_sold,cat_shop_gross
0,0,0,32,6.0,1,221.0,1326.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,299.0,101515.86,33489.0,8667236.69,1838.0,308301.0
1,0,0,33,3.0,1,347.0,1041.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,61.0,29831.00,6094.0,2845049.46,302.0,93622.0
2,0,0,35,1.0,1,247.0,247.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,78.0,30210.00,33489.0,8667236.69,1838.0,308301.0
3,0,0,43,1.0,1,221.0,221.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,2.0,469.00,33489.0,8667236.69,1838.0,308301.0
4,0,0,51,2.0,1,128.5,257.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,13.0,2996.00,1539.0,427204.87,60.0,9355.0
5,0,0,61,1.0,1,195.0,195.0,1000 и одна ночь 2CD (mp3-DVD) (DVD-Book),43,Книги - Аудиокниги,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,2.0,494.00,1352.0,406827.80,53.0,9349.0
6,0,0,75,1.0,1,76.0,76.0,12 ДРУЗЕЙ ОУШЕНА WB (регион),40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,8.0,1089.20,33489.0,8667236.69,1838.0,308301.0
7,0,0,88,1.0,1,76.0,76.0,13 ДРУЗЕЙ ОУШЕНА WB (регион),40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,7.0,940.20,33489.0,8667236.69,1838.0,308301.0
8,0,0,95,1.0,1,193.0,193.0,1812. ЭНЦИКЛОПЕДИЯ ВЕЛИКОЙ ВОЙНЫ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,7.0,1775.00,33489.0,8667236.69,1838.0,308301.0
9,0,0,96,1.0,1,70.0,70.0,1812. ЭНЦИКЛОПЕДИЯ ВОЙНЫ (регион),40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",5578.0,2966412.0,9.0,1262.00,33489.0,8667236.69,1838.0,308301.0


Now, this data is a very denormalized representation of the training data (because it was enriched with e.g. the aggregate sales data of items in the same category globally or in the same shop *in the same month*).  Unfortunately, for our prediction purposes, this equivalent data is not available (it would need to be predicted as well, then aggregated; this is too complex at this stage)... so we will time-shift the data and join it onto the training (and test data), so that the prediction problem gets information about the past (moving average style).  There are some arbitrary decisions about what features we want to add: let's use `past_months` data (not too many to avoid losing too many rows, i.e. those for which no past data is known) and the sum of the same month over the whole data range (this expresses the assumption that there's a yearly pattern).

Note that we work with the sum instead of the median to sidestep the problem of missing values (a product sold only in the first year in a given shop is simply absent instead of having a row with `item_cnt_day = 0`).  Using the mean (that is just a scale factor) would have the same problem

In [21]:
past_months = 4
aggregation_cols = (
    'item_cnt_day', 'gross', 'all_shop_items_sold',
    'all_shop_gross', 'global_sold', 'global_gross', 'global_cat_sold',
    'global_cat_gross', 'cat_shop_sold', 'cat_shop_gross')
data_monthly_sum = data.groupby(['Month', 'shop_id', 'item_id'], as_index=False).agg({
    k: 'sum' for k in aggregation_cols})
data_monthly_sum.rename(inplace=True, columns={k: "{}_sum".format(k) for k in aggregation_cols});
data_monthly_sum[data_monthly_sum['shop_id'] == 32].head(50)
#sales_train[(sales_train['item_id'] == 32) & (sales_train['date_block_num'] %12 == 1)]

,Month,shop_id,item_id,item_cnt_day_sum,gross_sum,all_shop_items_sold_sum,all_shop_gross_sum,global_sold_sum,global_gross_sum,global_cat_sold_sum,global_cat_gross_sum,cat_shop_sold_sum,cat_shop_gross_sum
78757,1,32,32,1.0,349.0,1238.0,824625.0,299.0,101515.86,33489.0,8667236.69,295.0,70460.6
78758,1,32,33,1.0,499.0,1238.0,824625.0,61.0,29831.00,6094.0,2845049.46,62.0,33567.0
78759,1,32,35,1.0,399.0,1238.0,824625.0,78.0,30210.00,33489.0,8667236.69,295.0,70460.6
78760,1,32,72,1.0,95.2,1238.0,824625.0,4.0,452.20,33489.0,8667236.69,295.0,70460.6
78761,1,32,75,1.0,119.2,1238.0,824625.0,8.0,1089.20,33489.0,8667236.69,295.0,70460.6
78762,1,32,85,0.0,0.0,1238.0,824625.0,17.0,4974.00,6094.0,2845049.46,62.0,33567.0
78763,1,32,88,1.0,119.2,1238.0,824625.0,7.0,940.20,33489.0,8667236.69,295.0,70460.6
78764,1,32,93,2.0,498.0,1238.0,824625.0,7.0,1743.00,1352.0,406827.80,23.0,7027.0
78765,1,32,98,2.0,798.0,1238.0,824625.0,361.0,137338.00,33489.0,8667236.69,295.0,70460.6
78766,1,32,153,1.0,399.0,1238.0,824625.0,12.0,4647.00,333.0,83313.00,6.0,1746.0


In [15]:
cat_vars = ['shop_id', 'item_id', 'item_name', 'item_category_id', 'item_category_name', 'shop_name', 'Year', 'Month']
cont_vars = ['item_price',
       'gross', 'open_days', 'all_shop_items_sold', 'all_shop_gross',
       'global_sold', 'global_gross', 'global_cat_sold', 'global_cat_gross',
       'cat_shop_sold', 'cat_shop_gross']

In [16]:
len(data.columns) - len(cat_vars) - len(cont_vars) - 2 # 2: dependent variable & time

0

Reorder data & cast to types expected by PyTorch (float32 for continuous variables, explicitly label categorical variables as such) (copied from lesson3-rossmann).

In [25]:
dep = 'item_cnt_day'

for v in cat_vars: 
    data[v] = data[v].astype('category').cat.as_ordered()

for v in cont_vars:
    data[v] = data[v].astype('float32')

data = data[cat_vars+cont_vars+[dep, 'date_block_num']].copy()

Todo:

  1. Embedding for item name & item category name
  2. Skip shop name?
  3. Drop 'gross' column: it is too correlated with 'item_cnt_day' and I don't have it in the test set
  4. Need to apply same enrichment to test data (i.e. add item categories)
  5. time series yet only data from the same month is used?  I don't even have it for the test data!  Incorporate results of previous months
  6. Define fitness function, clip output to [0, 20]

Select subset of data to speed up exploration

In [27]:
n = data.shape[0]

In [29]:
idxs = get_cv_idxs(n, val_pct=150000/n) 
joined_samp = data.iloc[idxs].set_index("date_block_num") 
samp_size = len(joined_samp); samp_size

150000

In [31]:
joined_samp.head(2)

,shop_id,item_id,item_name,item_category_id,item_category_name,shop_name,Year,Month,item_price,gross,open_days,all_shop_items_sold,all_shop_gross,global_sold,global_gross,global_cat_sold,global_cat_gross,cat_shop_sold,cat_shop_gross,item_cnt_day
date_block_num,,,,,,,,,,,,,,,,,,,,
17,38,2196,COLDPLAY Ghost Stories,55,Музыка - CD локального производства,"Омск ТЦ ""Мега""",2014,6,299.0,598.0,30.0,1467.0,1533191.0,145.0,43012.449219,10769.0,3077388.25,126.0,35179.00000,2.0
19,31,9964,ВОЗДУШНЫЙ МАРШАЛ,40,Кино - DVD,"Москва ТЦ ""Семеновский""",2014,8,399.0,2793.0,31.0,8248.0,5763062.5,93.0,36641.898438,15821.0,4307263.50,1841.0,474910.09375,7.0


In [36]:
df, y, nas, mapper = proc_df(joined_samp, dep, do_scale=True, skip_flds=['gross'])
yl = np.log(y)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [37]:
sum(y == 0.0)

256

In [38]:
df.head(2)

,shop_id,item_id,item_name,item_category_id,item_category_name,shop_name,Year,Month,item_price,open_days,all_shop_items_sold,all_shop_gross,global_sold,global_gross,global_cat_sold,global_cat_gross,cat_shop_sold,cat_shop_gross
date_block_num,,,,,,,,,,,,,,,,,,
17,39,2152,2152,56,56,39,2,6,-0.324435,-0.054396,-0.721803,-0.610916,0.375586,-0.044211,0.168194,-0.316104,-0.401182,-0.542709
19,32,9776,9776,41,41,32,2,8,-0.258757,0.472730,1.852293,1.225618,0.149892,-0.058190,0.749037,-0.052203,3.097769,1.584466


In [39]:
val_idx = np.flatnonzero(df.index == max(df.index))

In [40]:
val_idx.shape

(2956,)

In [41]:
n

1609124

In [17]:
test.head(5)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


The [competition page](https://www.kaggle.com/c/competitive-data-science-predict-future-sales#evaluation) says to use RMSE as metric:

In [1]:
def inv_y(a): return np.exp(a)

def exp_rmse(y_pred, targ):
    targ = inv_y(targ)
    pct_var = ()/targ
    return math.sqrt(((targ - inv_y(y_pred))**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

NameError: name 'np' is not defined